In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))
sp.init_printing()

In [ ]:
import datetime
import networkx as nx
from random import shuffle

from pydivest.macro_model.integrate_equations_mean import IntegrateEquationsMean as mean_model
from pydivest.macro_model.integrate_equations_aggregate import IntegrateEquationsAggregate as aggregate_model

# investment_decisions:

nopinions = [50, 50]

# Parameters:

phi, b_d = 0.4, 1.25

input_parameters = {'b_c': 1., 'phi': phi, 'tau': 1.,
                    'eps': 0.05, 'b_d': b_d, 'e': 100.,
                    'b_r0': 0.1 ** 2 * 100.,
                    'possible_opinions': [[0], [1]],
                    'xi': 1. / 8., 'beta': 0.06,
                    'L': 100., 'C': 100., 'G_0': 800.,
                    'campaign': False, 'learning': True,
                    'R_depletion': False, 'test': True,
                    'interaction': 1, 'crs': True}

# investment_decisions
opinions = []
for i, n in enumerate(nopinions):
    opinions.append(np.full((n), i, dtype='I'))
opinions = [item for sublist in opinions for item in sublist]
shuffle(opinions)

# network:
N = sum(nopinions)
p = .2

while True:
    net = nx.erdos_renyi_graph(N, p)
    if len(list(net)) > 1:
        break
adjacency_matrix = nx.adj_matrix(net).toarray()

# investment
clean_investment = np.ones(N)
dirty_investment = np.ones(N)

init_conditions = (adjacency_matrix, opinions,
                   clean_investment, dirty_investment)

In [ ]:
mm = mean_model(*init_conditions, **input_parameters)
mm.run(t_max=200)
mm.R_depletion = True
mm.run(t_max=600)
trj_m = mm.get_unified_trajectory()

In [ ]:
ma = aggregate_model(*init_conditions, **input_parameters)
ma.run(t_max=200)
ma.R_depletion = True
ma.run(t_max=600)
trj_a = ma.get_unified_trajectory()

In [ ]:
for c in trj_m.columns:
    dif = trj_m[[c]] - trj_a[[c]]
    dif_cum = dif.cumsum()
    max_dif = dif_cum.max()
    print(max_dif)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ma.get_aggregate_trajectory()[['K_c^c', 'K_c^d', 'K_d^c', 'K_d^d']].plot(ax=ax)

In [ ]:
trj_a.columns

In [ ]:
trj = mm.get_mean_trajectory()

In [ ]:
trj[['mu_c^c', 'mu_d^d']].plot()